In [1]:
import requests
import json
import gzip
import pandas as pd
from io import BytesIO
import copy
from requests.exceptions import ReadTimeout, ConnectionError, Timeout
import time
import re
import os

In [44]:
def make_request(url):
    attempt = 0
    backoff = 5
    
    while attempt < 10:
        try:
            response = requests.get(url, timeout=10)
            return response
        except (ReadTimeout, Timeout) as e:
            attempt += 1
            time.sleep(backoff)
            backoff = min(backoff * 2, 60)
        except requests.RequestException as e:
            print(f"An error occurred: {e}")
            break
    
    print("Max retries reached. Request failed.")
    return None

esports_data_files="leagues", "tournaments", "players", "teams", "mapping_data"

In [45]:
def get_esport_data_file(file, league):
    S3_BUCKET_URL = f"https://vcthackathon-data.s3.amazonaws.com/{league}"
    file_name = f"esports-data/{file}"

    response = make_request(f"{S3_BUCKET_URL}/{file_name}.json.gz")
    if response.status_code == 200:
        gzip_bytes = BytesIO(response.content)
        with gzip.GzipFile(fileobj=gzip_bytes, mode="rb") as gzipped_file:
            json_item = gzipped_file.read()

        json_string = json_item.decode('utf-8')
        return json.loads(json_string)
    else:
        raise ConnectionError(f'Bad response code: {response.status_code}')

In [46]:
def filter_game_data(game_data):
    game_dic = {
        'configuration': [],
        'playerDied': [],
        'gameDecided': [],
        'damageEvent': []
    }
    config = False
    for item in game_data:
        if 'configuration' in item.keys() and config == False:
            config = True
            game_dic['configuration'].append(item)
        
        if 'playerDied' in item.keys() or 'roundEnded' in item.keys():
            game_dic['playerDied'].append(item)
        
        if 'gameDecided' in item.keys():
            game_dic['gameDecided'].append(item)

        if 'damageEvent' in item.keys():
            game_dic['damageEvent'].append(item)
            
    return game_dic

In [47]:
def save_game_data(response, league, game_map):
    if league == 'vct-international':
        directory = 'vct'
    if league == 'vct-challengers':
        directory = 'vcl'
    if league == 'game-changers':
        directory = 'gc'
    
    gzip_bytes = BytesIO(response.content)
    with gzip.GzipFile(fileobj=gzip_bytes, mode="rb") as gzipped_file:
        json_item = gzipped_file.read()

    json_string = json_item.decode('utf-8')
    json_data = json.loads(json_string)
    game_data = filter_game_data(game_data=json_data)

    team_map = game_map['teamMapping']
    player_map = game_map['participantMapping']

    configuration = game_data['configuration'][0]

    match_id = configuration['platformGameId']
    date =  configuration['metadata']['wallTime']

    try:
        game_decided = game_data['gameDecided'][0]
    except IndexError:
        game_decided = None
        rounds = None
        win = None

    if game_decided:
        rounds = game_decided['gameDecided']['spikeMode']['currentRound']
        try:
            win = game_decided['gameDecided']['winningTeam']['value']
        except KeyError:
            win = None
    
    stats = {
        'match_id': match_id,
        'date': date,
        'total_rounds': rounds if rounds else None,
        'winning_team': team_map[str(win)] if win else None
    }

    player_stats = {
        i: copy.deepcopy(stats) for i in range(1, 11)
    }

    red_team = configuration['configuration']['teams'][0]
    blue_team = configuration['configuration']['teams'][1]
    
    red_id = red_team['teamId']['value']
    blue_id = blue_team['teamId']['value']

    red_players = [player['value'] for player in red_team['playersInTeam']]
    for player in red_players:
        player_stats[player]['player_id'] = player_map[str(player)]
        player_stats[player]['team_id'] = team_map[str(red_id)]
    
    blue_players = [player['value'] for player in blue_team['playersInTeam']]
    for player in blue_players:
        player_stats[player]['player_id'] = player_map[str(player)]
        player_stats[player]['team_id'] = team_map[str(blue_id)]

    for player in configuration['configuration']['players']:
        identifier = player['playerId']['value']
        agent_id = player['selectedAgent']['fallback']['guid']
        player_stats[identifier]['agent_id'] = agent_id

        player_name = player['displayName']
        player_stats[identifier]['display_name'] = player_name
    
    
    player_died = game_data['playerDied']
    round = 'No rounds'
    current_round = 1
    for event in player_died:
        if 'roundEnded' in event.keys():
            current_round = event['roundEnded']['roundNumber'] + 1
            continue
        
        try:
            deceased_id = event['playerDied']['deceasedId']['value']
            killer_id = event['playerDied']['killerId']['value']
        except KeyError:
            continue
        
        try:
            hazard = event['playerDied']['hazard']
            if hazard['type'] == 'SPIKE':
                continue
        except KeyError:
            pass

        if (deceased_id in red_players and killer_id in blue_players) or (deceased_id in blue_players and killer_id in red_players):
            player_stats[killer_id]['kills'] = player_stats[killer_id].get('kills', 0) + 1
            player_stats[deceased_id]['deaths'] = player_stats[deceased_id].get('deaths', 0) + 1

            for a in event['playerDied']['assistants']:
                assistant_id = a['assistantId']['value']
                player_stats[assistant_id]['assists'] = player_stats[assistant_id].get('assists', 0) + 1
            
            if round == 'No rounds' or round < current_round:
                round = current_round
                player_stats[killer_id]['first_kills'] = player_stats[killer_id].get('first_kills', 0) + 1
                player_stats[deceased_id]['first_deaths'] = player_stats[deceased_id].get('first_deaths', 0) + 1
            
    damage_event = game_data['damageEvent']
    for event in damage_event:
        try:
            causer_id = event['damageEvent']['causerId']['value']
            victim_id = event['damageEvent']['victimId']['value']
            damage_dealt = event['damageEvent']['damageDealt']
        except KeyError:
            continue
            

        if (causer_id in red_players and victim_id in blue_players) or (causer_id in blue_players and victim_id in red_players):
            try:
                is_hazard = event['damageEvent']['hazard']
            except KeyError:
                is_hazard = None
                pass
            if is_hazard:
                hazard = event['damageEvent']['hazard']
                if hazard['type'] == 'SPIKE':
                    continue
                if 'Decay' in hazard['fallback']['displayName']:
                    continue

                player_stats[causer_id]['damage'] = player_stats[causer_id].get('damage', 0) + damage_dealt
              
            
            try:
                is_weapon = event['damageEvent']['weapon']
            except KeyError:
                is_weapon = None
                pass
            if is_weapon:
                player_stats[causer_id]['shots'] = player_stats[causer_id].get('shots', 0) + 1
                player_stats[causer_id]['damage'] = player_stats[causer_id].get('damage', 0) + damage_dealt
                if event['damageEvent']['location'] == 'HEAD':
                    player_stats[causer_id]['headshots'] = player_stats[causer_id].get('headshots', 0) + 1
                
            try:
                is_ability = event['damageEvent']['ability']
            except KeyError:
                is_ability = None
                pass
            if is_ability:
                player_stats[causer_id]['damage'] = player_stats[causer_id].get('damage', 0) + damage_dealt
    
    with open(f'data/games/{directory}/{re.sub(r'[<>:"/\\|?*]', '_', match_id).strip()}.json', 'w') as f:
        json.dump(player_stats, f)

In [48]:
def filter_mapping_data(year, league, game_map):
    S3_BUCKET_URL = f"https://vcthackathon-data.s3.amazonaws.com/{league}"
    file_name = f"games/{year}/{game_map['platformGameId']}"

    response = make_request(f"{S3_BUCKET_URL}/{file_name}.json.gz")
    if response.status_code == 200:
        save_game_data(response=response, league=league, game_map=game_map)
        return True
    else:
        return False, None

In [7]:
filtered_mapping_data_dic = {
    1001: [],
    2002: [],
    3003: []
}

for league in ['vct-international', 'vct-challengers', 'game-changers']:
    if league == 'vct-international':
        year_list = [2022, 2023, 2024]
        dir = 'vct'
        league_id = 1001
    if league == 'vct-challengers':
        year_list = [2023, 2024]
        dir = 'vcl'
        league_id = 2002
    if league == 'game-changers':
        year_list = [2022, 2023, 2024]
        dir = 'gc'
        league_id = 3003
    mapping_data = get_esport_data_file(file='mapping_data', league=league)
    
    for game_map in mapping_data:
        if os.path.exists(f'data/games/{dir}/{re.sub(r'[<>:"/\\|?*]', '_', game_map['platformGameId']).strip()}.json'):
            filtered_mapping_data_dic[league_id].append(game_map)
            continue
        for year in year_list:
            is_game = filter_mapping_data(year=year, league=league, game_map=game_map)
            if is_game == True:
                filtered_mapping_data_dic[league_id].append(game_map)
                break
    
with open(f'data/mapping_data/mapping_data.json', 'w') as f:
    json.dump(filtered_mapping_data_dic, f)

In [40]:
with open('data/mapping_data/mapping_data.json', 'r') as f:
    map_data = json.load(f)

df = []
for league in ['1001', '2002', '3003']:
    match_list = map_data[league]
    if league == '1001':
        dir = 'vct'
    if league == '2002':
        dir = 'vcl'
    if league == '3003':
        dir = 'gc'
    
    for item in match_list:
        with open(f"data/games/{dir}/{re.sub(r'[<>:"/\\|?*]', '_', item['platformGameId']).strip()}.json") as f:
            game_data = json.load(f)
        for player in game_data.values():
            df.append({
                'player_id': player['player_id'],
                'team_id': player['team_id'],
                'date': player['date'],
                'tournament_id': item['tournamentId'],
                'league_id': league
            })

In [41]:
df = pd.DataFrame(df)

In [42]:
# Region ids Americas: 1, EMEA: 2, China: 3, Pacific: 4, International: 7
regions = {'109940824119741550': 7, '109974804058058602': 4, '109551178413356399': 7,
        '106109559530232966': 2, '111691194187846945': 3, '109974795266458277': 1,
        '107254585505459304': 7, '105555627532605797': 4, '106470453892538426': 4,
        '105555705801095792': 4, '105555608835603034': 1, '106132846649518478': 1,
        '105555635175479654': 1, '109551156421687480': 2, '105555664141146477': 1,
        '105555704030157191': 4, '110032750555964337': 4, '109551158912464238': 2,
        '105555699868690469': 4, '105555678532655472': 4, '105555666330314783': 1,
        '105555707205380136': 4, '107910334624279390': 4, '105555677198805024': 4,
        '109029777807406730': 4, '112609302000026507': 4, '108752229027041361': 4,
        '106375817979489820': 1, '109879575186437267': 1, '106976737954740691': 1,
        '107019646737643925': 2, '113005438924105895': 4, '109222784797127274': 7,
        '109109622360706601': 4, '107115678205203231': 4
    }

In [49]:
tournaments_dic = {
    1001: [],
    2002: [],
    3003: []
}
with open(f'data/mapping_data/mapping_data.json', 'r') as f:
    mapping_data = json.load(f)

for league in ['vct-international', 'vct-challengers', 'game-changers']:
    if league == 'vct-international':
        league_id = 1001
    if league == 'vct-challengers':
        league_id = 2002
    if league == 'game-changers':
        league_id = 3003
    
    tournament_list = {map['tournamentId'] for map in mapping_data[str(league_id)]}

    tournament_data = get_esport_data_file(file='tournaments', league=league)
    
    for item in tournament_data:
        item['region_id'] = regions[item['league_id']]
        if item['id'] in tournament_list:
            tournaments_dic[league_id].append(item)
with open(f'data/tournaments/tournaments.json', 'w') as f:
    json.dump(tournaments_dic, f)

In [50]:
with open(f'data/tournaments/tournaments.json', 'r') as f:
    tournament_data = json.load(f)
tournament_df = []
for league in ['1001', '2002', '3003']:
    for item in tournament_data[league]:
        tournament_df.append({
            'tournament_id': item['id'],
            'region_id': item['region_id'],
            'league_id': league
        })
tournament_df = pd.DataFrame(tournament_df)

In [51]:
df = pd.merge(
    left=df,
    right=tournament_df,
    on=['tournament_id', 'league_id'],
    how='left'
)

In [53]:
df['date'] = pd.to_datetime(df['date'], format='ISO8601')
df = df[(df['player_id'] != '-') & (df['team_id'] != '-')].reset_index(drop=True)

players = df.sort_values(by=['player_id', 'date']).reset_index(drop=True)

players['previous_team'] = players['team_id'] != players['team_id'].shift()
players['previous_player'] = players['player_id'] != players['player_id'].shift()


team_change_row = []
for index, row in players.iterrows():
    if row['previous_player'] == True:
        team_change = 1
    elif row['previous_team'] == True:
        team_change += 1
    team_change_row.append(team_change)
players['team_change'] = team_change_row

In [168]:
player_movements = players.groupby(['player_id', 'team_id', 'team_change']).agg(
    start_date=('date', 'min'),
    end_date=('date', 'max')
).reset_index()

max_end_date_idx = player_movements.groupby('player_id')['end_date'].idxmax()
for idx in max_end_date_idx:
    if player_movements.loc[idx, 'end_date'].year == 2024:
        player_movements.loc[idx, 'end_date'] = None

In [169]:
player_movements['start_date'] = player_movements['start_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
player_movements['end_date'] = player_movements['end_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
with open('data/players/player_movements.json', 'w') as f:
    json.dump(player_movements.to_dict(orient='records'), f)

In [170]:
teams = df[(df['region_id'] != 7)].reset_index(drop=True)
teams = teams.sort_values(by=['team_id', 'date']).reset_index(drop=True)

teams['previous_team'] = teams['team_id'] != teams['team_id'].shift()
teams['previous_league'] = teams['league_id'] != teams['league_id'].shift()
teams['previous_region'] = teams['region_id'] != teams['region_id'].shift()

In [171]:
region_change_row = []
for index, row in teams.iterrows():
    if row['previous_team'] == True:
        team_change = 1
    elif row['previous_league'] == True:
        team_change += 1
    elif row['previous_region'] == True:
        if row['region_id'] == 7:
            team_change=team_change
        else:
            team_change += 1
    region_change_row.append(team_change)
teams['team_change'] = region_change_row

In [172]:
teams = teams.groupby(['team_id', 'league_id', 'region_id', 'team_change']).agg(
    start_date=('date', 'min'),
    end_date=('date', 'max')
).reset_index()

max_end_date_idx = teams.groupby('team_id')['end_date'].idxmax()
for idx in max_end_date_idx:
    if teams.loc[idx, 'end_date'].year == 2024:
        teams.loc[idx, 'end_date'] = None

In [173]:
teams['start_date'] = teams['start_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
teams['end_date'] = teams['end_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
with open('data/teams/team_movements.json', 'w') as f:
    json.dump(teams.to_dict(orient='records'), f)

In [101]:
teams = []
for league in ['vct-international', 'vct-challengers', 'game-changers']:
    teams.extend(item for item in get_esport_data_file(file='teams', league=league))

In [102]:
teams = pd.DataFrame(teams)
teams = teams[['id', 'name', 'acronym']].drop_duplicates(subset='id', ignore_index=True)

In [103]:
with open('data/teams/team_movements.json', 'r') as f:
    team_movements = json.load(f)

In [104]:
team_movements = pd.DataFrame(team_movements)
current_data = team_movements.loc[team_movements['end_date'].isna()][['team_id', 'league_id', 'region_id']]

In [105]:
df = pd.merge(
    left=teams,
    right=current_data,
    left_on='id',
    right_on='team_id',
    how='left'
)
df = df[['id', 'name', 'acronym', 'league_id', 'region_id']].rename(columns={'id': 'team_id', 'league_id': 'home_league', 'region_id': 'home_region'})

In [107]:
with open('data/teams/teams.json', 'w') as f:
    json.dump(df.to_dict(orient='records'), f)

In [228]:
players = []
for league in ['vct-international', 'vct-challengers', 'game-changers']:
    players.extend(item for item in get_esport_data_file(file='players', league=league))
players = pd.DataFrame(players)

In [229]:
players = players[['id', 'handle', 'first_name', 'last_name']].rename(columns={'id': 'player_id'})
players = players.drop_duplicates(subset='player_id', ignore_index=True)

In [230]:
with open('data/players/player_movements.json', 'r') as f:
    player_movements = json.load(f)

In [231]:
filter_ids = {item['player_id'] for item in player_movements}
players = players[players['player_id'].isin(filter_ids)].reset_index(drop=True)

In [233]:
player_movements = pd.DataFrame(player_movements)
current_data = player_movements.loc[player_movements['end_date'].isna()][['player_id', 'team_id']]

In [237]:
df = pd.merge(
    left=players,
    right=current_data,
    on = 'player_id',
    how='left'
).rename(columns={'team_id': 'home_team'})

In [238]:
with open('data/players/players.json', 'w') as f:
    json.dump(df.to_dict(orient='records'), f)

In [9]:
with open('data/mapping_data/mapping_data.json', 'r') as f:
    map_data = json.load(f)

In [242]:
df = []
for league in ['1001', '2002', '3003']:
    if league == '1001':
        dir = 'vct'
    if league == '2002':
        dir = 'vcl'
    if league == '3003':
        dir = 'gc'
    for item in map_data[league]:
        with open(f'data/games/{dir}/{re.sub(r'[<>:"/\\|?*]', '_', item['platformGameId']).strip()}.json', 'r') as f:
            match_data = json.load(f)

        match_data = match_data['1']

        df.append({
            'platform_game_id': item['platformGameId'],
            'match_id': item['esportsGameId'],
            'tournament_id': item['tournamentId'],
            'rounds': match_data['total_rounds'],
            'date': match_data['date'],
            'league': league
        })

In [247]:
matches = pd.DataFrame(df)

In [246]:
with open(f'data/tournaments/tournaments.json', 'r') as f:
    tournament_data = json.load(f)
tournament_df = []
for league in ['1001', '2002', '3003']:
    for item in tournament_data[league]:
        tournament_df.append({
            'tournament_id': item['id'],
            'region_id': item['region_id'],
            'league_id': league
        })
tournament_df = pd.DataFrame(tournament_df)

In [252]:
df = pd.merge(
    left=matches,
    right=tournament_df,
    on='tournament_id',
    how='left'
)
with open('data/matches/matches.json', 'w') as f:
    json.dump(df.to_dict(orient='records'), f)

In [10]:
player_stats = []
for league in ['1001', '2002', '3003']:
    if league == '1001':
        dir = 'vct'
    if league == '2002':
        dir = 'vcl'
    if league == '3003':
        dir = 'gc'    
    for item in map_data[league]:
        with open(f'data/games/{dir}/{re.sub(r'[<>:"/\\|?*]', '_', item['platformGameId']).strip()}.json', 'r') as f:
            raw_player_stats = json.load(f)
        
        for player in raw_player_stats.values():
            if player['player_id'] == '-':
                continue
            player['match_id'] = item['platformGameId']
            if 'damage' in player.keys() and 'total_rounds' in player.keys():
                if player['damage'] == None or player['total_rounds'] == None:
                    damage_per_round = None
                else:
                    damage_per_round = round(player['damage'] / player['total_rounds'], 2)
            else:
                damage_per_round = None
            if 'headshots' in player.keys() and 'shots' in player.keys():
                headshot_percent = round(player['headshots'] / player['shots'], 2)
            else:
                headshot_percent = None
                   

            player_stats.append({
                'player_id': player['player_id'],
                'match_id': player['match_id'],
                'agent_id': player['agent_id'],
                'kills': player.get('kills', 0),
                'deaths': player.get('deaths', 0),
                'assists': player.get('assists', 0),
                'avg_dmg_rnd': damage_per_round,
                'headshot_percent': headshot_percent, 
                'first_kills': player.get('first_kills', 0),
                'first_deaths': player.get('first_deaths', 0)
            })
            
            

In [11]:
player_stats_df = pd.DataFrame(player_stats)

In [12]:
with open('data/matches/matches.json', 'r') as f:
    matches = json.load(f)

In [13]:
matches_df = pd.DataFrame(matches)

league_mapping = {
    '1001': 'VCT International',
    '2002': 'VCT Challengers',
    '3003': 'VCT Game Changers'
}

matches_df['league_name'] = matches_df['league_id'].map(league_mapping)

region_mapping = {
    1: 'Americas',
    2: 'EMEA',
    3: 'China',
    4: 'Pacific',
    7: 'International'
}

matches_df['region_name'] = matches_df['region_id'].map(region_mapping)

matches_df = matches_df[['platform_game_id', 'date', 'league_name', 'region_name']].rename(columns={'platform_game_id': 'match_id'})
matches_df['date'] = pd.to_datetime(matches_df['date'], format='ISO8601')

In [14]:
player_stats_df = pd.merge(
    left=player_stats_df,
    right=matches_df,
    on='match_id',
    how='left'
)

In [16]:
import psycopg2
conn = psycopg2.connect(
    dbname="vct",
    user="postgres",
    password="122357",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

sql_query = "SELECT * FROM agents;"

cur.execute(sql_query)

rows = cur.fetchall()

cur.close()
conn.close()

In [17]:
agents = pd.DataFrame(rows, columns=['agent_id', 'agent_name', 'agent_role'])

In [18]:
player_stats_df = pd.merge(
    left=player_stats_df,
    right=agents,
    on='agent_id',
    how='left'
)

In [20]:
player_stats_df['rating'] = 0.2*player_stats_df['kills'] - 0.2*player_stats_df['deaths'] + 0.1*player_stats_df['assists'] + 0.25*player_stats_df['first_kills'] - 0.25*player_stats_df['first_deaths']
def normalize_ratings(group):
    min_rating = group['rating'].min()
    max_rating = group['rating'].max()
    group['normalized_rating'] = round((group['rating'] - min_rating) / (max_rating - min_rating), 2)
    return group
rating_df = player_stats_df.groupby(['league_name', 'agent_role']).apply(normalize_ratings, include_groups=False)[['player_id', 'match_id', 'normalized_rating']].reset_index(drop=True)

In [21]:
player_stats_df = pd.merge(
    left=player_stats_df,
    right=rating_df,
    on=['player_id', 'match_id'],
    how='inner'
)

In [189]:
player_stats_df = pd.read_csv('data/complete_data/full_player_stats.csv')
player_stats_df = player_stats_df.sort_values('date', ascending=False)

In [190]:
import numpy as np
def decay_weighted_average(values, decay=0.9):
    n = len(values)

    weights = np.array([decay**i for i in range(n)])
    
    return np.sum(weights * values) / np.sum(weights)

In [191]:
decay_factor = 0.75
rating = player_stats_df.groupby('player_id')['normalized_rating'].apply(lambda x: decay_weighted_average(x, decay=decay_factor)).reset_index(name='total_rating')

In [192]:
totals = player_stats_df.groupby('player_id').agg(
    total_games = ('match_id', 'count'),
    total_kills=('kills', 'sum'),
    total_deaths=('deaths', 'sum'),
    total_assists=('assists', 'sum'),
    total_first_kills=('first_kills', 'sum'),
    total_first_deaths=('first_deaths', 'sum'),
    total_headshot=('headshot_percent', 'mean'),
    total_avg_dmg_rnd=('avg_dmg_rnd', 'mean'),
).reset_index()

In [193]:
totals = pd.merge(
    left=totals,
    right=rating,
    on='player_id',
    how='left'
)

In [194]:
decay_factor = 0.75
rating = player_stats_df[player_stats_df['region_name'].isin(['International'])].groupby('player_id')['normalized_rating'].apply(lambda x: decay_weighted_average(x, decay=decay_factor)).reset_index(name='international_rating')

In [195]:
international = player_stats_df[player_stats_df['region_name'].isin(['International'])].groupby('player_id').agg(
    international_games = ('match_id', 'count'),
    international_kills=('kills', 'sum'),
    international_deaths=('deaths', 'sum'),
    international_assists=('assists', 'sum'),
    international_first_kills=('first_kills', 'sum'),
    international_first_deaths=('first_deaths', 'sum'),
    international_headshot=('headshot_percent', 'mean'),
    international_avg_dmg_rnd=('avg_dmg_rnd', 'mean'),
).reset_index()

In [196]:
international = pd.merge(
    left=international,
    right=rating,
    on='player_id',
    how='left'
)

In [197]:
regional = player_stats_df[player_stats_df['region_name'].isin(['Americcas', 'EMEA', 'China', 'Pacific'])].groupby('player_id').agg(
    regional_games = ('match_id', 'count'),
    regional_rating = ('normalized_rating', 'mean'),
    regional_kills=('kills', 'sum'),
    regional_deaths=('deaths', 'sum'),
    regional_assists=('assists', 'sum'),
    regional_first_kills=('first_kills', 'sum'),
    regional_first_deaths=('first_deaths', 'sum'),
    regional_headshot=('headshot_percent', 'mean'),
    regional_avg_dmg_rnd=('avg_dmg_rnd', 'mean'),
).reset_index()

In [198]:
role_counts = player_stats_df.groupby(['player_id', 'agent_role']).size().reset_index(name='role_count')

total_matches = player_stats_df.groupby('player_id').size().reset_index(name='total_matches')

role_percentage = pd.merge(role_counts, total_matches, on='player_id')

role_percentage['percent_played'] = (role_percentage['role_count'] / role_percentage['total_matches']) * 100

In [199]:
role_pivot = role_percentage.pivot(index='player_id', columns='agent_role', values='percent_played').fillna(0)
def determine_role(row):
    if (row.max() > 55):
        return row.idxmax()
    elif (row >= 25).sum() >= 2:
        return 'Flex'
    else:
        return row.idxmax()

role_pivot['main_role'] = role_pivot.apply(determine_role, axis=1)
role_pivot = role_pivot.reset_index()

In [200]:
with open('data/players/players.json') as f:
    players = json.load(f)

In [201]:
players_df = pd.DataFrame(players)
players_df = players_df[players_df['home_team'].notna()]

In [202]:
with open('data/teams/teams.json') as f:
    teams = pd.DataFrame(json.load(f))

In [203]:
players_df = pd.merge(
    left=players_df,
    right=teams[['team_id', 'name', 'home_league', 'home_region']],
    left_on='home_team',
    right_on='team_id',
    how='left'
)[['player_id', 'handle', 'first_name', 'last_name', 'home_league', 'home_region', 'name']].rename(columns={'home_league':'league', 'home_region':'region', 'name':'team_name'})

In [204]:
role_pivot['player_id'] = role_pivot['player_id'].astype(str)
players_df = pd.merge(
    left=players_df,
    right=role_pivot[['player_id', 'main_role']],
    on='player_id',
    how='left'
).rename(columns={'main_role':'agent_role'})

In [205]:
totals['player_id'] = totals['player_id'].astype(str)
players_df = pd.merge(
    left=players_df,
    right=totals,
    on='player_id',
    how='left'
)

In [206]:
international['player_id'] = international['player_id'].astype(str)
players_df = pd.merge(
    left=players_df,
    right=international,
    on='player_id',
    how='left'
).fillna(0)

In [207]:
players_df[[
    'international_games', 
    'international_kills', 
    'international_deaths', 
    'international_assists',
    'international_first_kills',
    'international_first_deaths'
]] = players_df[[
        'international_games',
        'international_kills', 
        'international_deaths', 
        'international_assists',
        'international_first_kills',
        'international_first_deaths'
    ]].astype(int)

In [2]:
df = pd.read_csv('data/complete_data/weighted_players.csv')

In [4]:
test = df[(df['league'] == 'VCT International') & (df['total_games'] > 10) & (df['total_rating'] > .45)].sort_values(['agent_role', 'total_games'], ascending=[True, False])

In [18]:
test[test['agent_role'] == 'Sentinel'].head(5)

,player_id,handle,first_name,last_name,league,region,team_name,agent_role,total_games,total_kills,...,total_rating,international_games,international_kills,international_deaths,international_assists,international_first_kills,international_first_deaths,international_headshot,international_avg_dmg_rnd,international_rating
14,107764993888872529,Alfajer,Emir Ali,Beder,VCT International,EMEA,FNATIC,Sentinel,161,2698,...,0.501987,83,1402,1140,354,196,162,0.301591,150.292500,0.498985
253,108329605365228616,Meteor,Kim,Taeoh,VCT International,Pacific,Gen.G,Sentinel,151,2507,...,0.492360,51,894,750,170,138,116,0.311951,154.195122,0.488595
3,108395376345727088,Shyy,Fabian,Usnayo,VCT International,Americas,KRÜ Esports,Sentinel,106,1542,...,0.500198,30,461,437,206,62,51,0.275000,139.411250,0.507271
6,106652155439574920,nAts,Ayaz,Akhmetshin,VCT International,EMEA,Team Liquid,Sentinel,84,1397,...,0.467834,14,240,214,53,27,20,0.288333,137.792500,0.481892
295,107876440210025807,yetujey,Eray,Budak,VCT International,EMEA,FUT Esports,Sentinel,62,926,...,0.546423,19,277,269,78,46,41,0.247368,132.270000,0.520111
